<a href="https://colab.research.google.com/github/victoralcantara75/erythrocytes-classification/blob/develop/TCC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#math
import numpy as np
import sklearn
import skimage
import sklearn.model_selection

#ts and keras
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator

#visualization
import matplotlib.pyplot as plt

#utils
import os
import imageio

In [2]:
#dataset
!git clone https://github.com/victoralcantara75/train-test-erythrocytes.git

Cloning into 'train-test-erythrocytes'...
remote: Enumerating objects: 766, done.
remote: Counting objects: 100% (766/766), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 766 (delta 203), reused 766 (delta 203), pack-reused 0
Receiving objects: 100% (766/766), 731.72 KiB | 22.87 MiB/s, done.
Resolving deltas: 100% (203/203), done.


In [3]:
batch = 16
epochs = 30
opt = 'adam'
lr = 0.001

In [4]:
def loadDir(round):
  train_dir = './train-test-erythrocytes/dataset/5-fold/round_'+str(round)+'/train'
  test_dir = './train-test-erythrocytes/dataset/5-fold/round_'+str(round)+'/test'
  return train_dir, test_dir

In [5]:
def createModel():
  base_model = ResNet50(weights='imagenet', include_top=False)

  x = base_model.output
  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(1024, activation='relu')(x)
  predictions = keras.layers.Dense(3, activation='softmax')(x)

  model = keras.models.Model(inputs=base_model.input, outputs=predictions)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [6]:
def loadGenerator(train_dir, test_dir):

  train_datagen = ImageDataGenerator(
                          rescale=1. / 255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          vertical_flip=True,
                          horizontal_flip=True)
  test_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(train_dir, batch_size = batch)
  test_generator = test_datagen.flow_from_directory(test_dir, batch_size = batch)
  return train_generator, test_generator

In [7]:
accs = []

for i in range (1, 6):
  print("ROUND ", i)
  train_dir, test_dir = loadDir(i)
  model = createModel()
  train_imgs, test_imgs = loadGenerator(train_dir, test_dir)

  model.fit(train_imgs, epochs=epochs)
  result = model.evaluate(train_imgs)

  accs.append(result[1])

print("Result: ", np.mean(accs))

ROUND  1
94773248/94765736 [==============================] - 2s 0us/step
Found 500 images belonging to 3 classes.
Found 126 images belonging to 3 classes.
Epoch 1/30
32/32 [==============================] - 10s 304ms/step - loss: 1.1542 - accuracy: 0.6640
Epoch 2/30
32/32 [==============================] - 10s 297ms/step - loss: 0.5693 - accuracy: 0.7860
Epoch 3/30
32/32 [==============================] - 10s 300ms/step - loss: 0.4297 - accuracy: 0.8460
Epoch 4/30
32/32 [==============================] - 9s 296ms/step - loss: 0.3893 - accuracy: 0.8660
Epoch 5/30
32/32 [==============================] - 10s 298ms/step - loss: 0.3218 - accuracy: 0.8920
Epoch 6/30
32/32 [==============================] - 10s 300ms/step - loss: 0.3747 - accuracy: 0.8420
Epoch 7/30
32/32 [==============================] - 10s 299ms/step - loss: 0.3366 - accuracy: 0.8780
Epoch 8/30
32/32 [==============================] - 10s 299ms/step - loss: 0.2972 - accuracy: 0.8940
Epoch 9/30
32/32 [===================